In [44]:
import pandas as pd
import requests
from tqdm import tqdm_notebook
import json
import os
from sqlalchemy import create_engine
import numpy as np
import movies_api_key
import config_etl
import psycopg2

Import csv of imdb_id's

In [8]:
#read id csv
url = "https://raw.githubusercontent.com/hakbrk/ETL-Project/refs/heads/master/Data/unique_indmID.csv"
imdb_id = pd.read_csv(url)
print(imdb_id.head(5))

     imdb_id
0  tt0114709
1  tt0113497
2  tt0113228
3  tt0114885
4  tt0113041


In [9]:
imdb_id = imdb_id['imdb_id']

Create functions to parse data as API calls are made.  By doing this it reduces the need for itterows or additional for loops later in the code.

In [10]:
def writer_list(writer, imdb):
    final_list = []
    name_list = writer.split(',')
    for index in name_list:
        name = index.split(' (')[0]
        final_list.append({'imdb_id' : imdb,
                         'first_name' : name.rsplit(' ', 1)[0],
                         'last_name' : name.split()[-1]})
    return final_list

In [11]:
def process_list(name_string, imdb):
    final_list = []
    name_list = name_string.split(',')
    for index in name_list:
        final_list.append({'imdb_id' : imdb,
                               'first_name' : index.rsplit(' ', 1)[0],
                               'last_name' : index.split()[-1]})
    return final_list

API calls and parsing by use of functions

In [12]:
print(movies_api_key.api_key)

2b0e53c7


In [25]:
url = "http://www.omdbapi.com//?i="
api = f"&apikey={movies_api_key.api_key}"
actor_data = []
director_data = []
writer_data = []
awards_data = []
ratings_data = []

#instead of looping through all ids_ using (for id_ in tqdm_notebook(imdb_id), taking a list of 5
for id_ in ['tt0114709','tt0113497','tt0113228','tt0114885','tt0113041']:
    try:
        response = requests.get(url + id_+api).json()
    except JSONDecodeError:
        print(f'error at {id_}')
        response = requests.get(url + id_+api).json()
    try:
        director = response['Director']
        temp_list = process_list(director, id_)
        for item in temp_list:
             director_data.append(item)
        actor = response['Actors']
        temp_list = process_list(actor, id_)
        for item in temp_list:
             actor_data.append(item)
        writer = response['Writer']
        temp_list = writer_list(writer, id_)
        for item in temp_list:
             writer_data.append(item)
        rating = response['Rated']
        ratings_data.append({'imdb_id' : id_,
                             'ratings' : rating})
        award = response['Awards']
        awards_data.append({'imdb_id' : id_,
                            'awards' : award})
    except KeyError:
        print(f'KeyError at {id_}')
        director_data.append({'imdb_id' : id_,
                         'first_name' : 'No Data',
                         'last_name' : 'No Data'})
        actor_data.append({'imdb_id' : id_,
                         'first_name' : 'No Data',
                         'last_name' : 'No Data'})
        writer_data.append({'imdb_id' : id_,
                         'first_name' : 'No Data',
                         'last_name' : 'No Data'})
        awards_data.append({'imdb_id' : id_,
                            'awards' : 'No Data'})
        ratings_data.append({'imdb_id' : id_,
                             'ratings' : 'No Data'})

Convert list of dictionaries to DataFrames

In [26]:
actor_all_df = pd.DataFrame(actor_data)
writer_all_df = pd.DataFrame(writer_data)
director_all_df = pd.DataFrame(director_data)
ratings_all_df = pd.DataFrame(ratings_data)
awards_all_df = pd.DataFrame(awards_data)

Replace blanks left by individuals with only one name.

In [27]:
actor_all_df.replace('', 'N/A', inplace = True)
writer_all_df.replace('', 'N/A', inplace = True)
director_all_df.replace('', 'N/A', inplace = True)

Create individual DataFrames which will then be pushed to the database as tables.

In [28]:
film_actor_id_df = actor_all_df
actor_id = film_actor_id_df.groupby(['last_name', 'first_name']).ngroup()
film_actor_id_df['actor_id'] = actor_id
film_actor_id_df['actor_id'] = 'a' + film_actor_id_df['actor_id'].astype(str)
film_actor_df = film_actor_id_df.drop(['first_name', 'last_name'], axis=1)
film_actor_df.drop_duplicates(subset=['imdb_id', 'actor_id'], inplace = True)
actor_df = film_actor_id_df.drop(['imdb_id'], axis=1)
actor_df.drop_duplicates(subset='actor_id', inplace = True)

In [29]:
film_writer_id_df = writer_all_df
writer_id = film_writer_id_df.groupby(['last_name', 'first_name']).ngroup()
film_writer_id_df['writer_id'] = writer_id
film_writer_id_df['writer_id'] = 'w' + film_writer_id_df['writer_id'].astype(str)
film_writer_df = film_writer_id_df.drop(['first_name', 'last_name'], axis=1)
film_writer_df.drop_duplicates(subset=['imdb_id', 'writer_id'], inplace = True)
writer_df = film_writer_id_df.drop(['imdb_id'], axis=1)
writer_df.drop_duplicates(subset='writer_id', inplace = True)

In [30]:
film_director_id_df = director_all_df
director_id = film_director_id_df.groupby(['last_name', 'first_name']).ngroup()
film_director_id_df['director_id'] = director_id
film_director_id_df['director_id'] = 'd' + film_director_id_df['director_id'].astype(str)
film_director_df = film_director_id_df.drop(['first_name', 'last_name'], axis=1)
film_director_df.drop_duplicates(subset=['imdb_id', 'director_id'], inplace = True)
director_df = film_director_id_df.drop(['imdb_id'], axis=1)
director_df.drop_duplicates(subset='director_id', inplace = True)

In [31]:
film_ratings_df = ratings_all_df.drop_duplicates(subset=['imdb_id', 'ratings'])

In [32]:
film_awards_df = awards_all_df.drop_duplicates(subset=['awards', 'imdb_id'])

Confirm table structure and headers

In [34]:
film_actor_df.head()

,imdb_id,actor_id
0,tt0114709,a5
1,tt0114709,a0
2,tt0114709,a12
3,tt0113497,a14
4,tt0113497,a4


In [35]:
actor_df.head()

,first_name,last_name,actor_id
0,Tom,Hanks,a5
1,Tim,Allen,a0
2,Don,Rickles,a12
3,Robin,Williams,a14
4,Kirsten,Dunst,a4


In [36]:
film_writer_df.head()

,imdb_id,writer_id
0,tt0114709,w6
1,tt0114709,w1
2,tt0114709,w9
3,tt0113497,w4
4,tt0113497,w11


In [37]:
writer_df.head()

,first_name,last_name,writer_id
0,John,Lasseter,w6
1,Pete,Docter,w1
2,Andrew,Stanton,w9
3,Jonathan,Hensleigh,w4
4,Greg,Taylor,w11


In [38]:
film_director_df.head()

,imdb_id,director_id
0,tt0114709,d2
1,tt0113497,d1
2,tt0113228,d0
3,tt0114885,d4
4,tt0113041,d3


In [39]:
director_df.head()

,first_name,last_name,director_id
0,John,Lasseter,d2
1,Joe,Johnston,d1
2,Howard,Deutch,d0
3,Forest,Whitaker,d4
4,Charles,Shyer,d3


Below code can be used to call data from csv files to save time of making API calls

Initiate connection to database and verify tables present in database - insert your postgress username and password

In [47]:
#SQL login info
protocol = "postgresql"
url = "localhost"
port = 5432 # ur port might be 5432
db = "movies_db"
connection_string = f"{protocol}://{config_etl.username}:{config_etl.password}@{url}/{db}"
engine = create_engine(connection_string)


Push DataFrames to the database as tables

In [48]:
actor_df.to_sql(name='actor', con=engine, if_exists='append', index=False)
director_df.to_sql(name='director', con=engine, if_exists='append', index=False)
writer_df.to_sql(name='writer', con=engine, if_exists='append', index=False)
film_actor_df.to_sql(name='film_actor', con=engine, if_exists='append', index=False)
film_director_df.to_sql(name='film_director', con=engine, if_exists='append', index=False)
film_writer_df.to_sql(name='film_writer', con=engine, if_exists='append', index=False)
film_ratings_df.to_sql(name='film_rating', con=engine, if_exists='append', index=False)
film_awards_df.to_sql(name='film_awards', con=engine, if_exists='append', index=False)

5

Create csv files for later use if needed.

In [ ]:
# actor_df.to_csv('actor_df.csv', index=False)
# director_df.to_csv('director_df.csv', index=False)
# writer_df.to_csv('writer_df.csv', index=False)
# film_actor_df.to_csv('film_actor_df.csv', index=False)
# film_director_df.to_csv('film_director_df.csv', index=False)
# film_writer_df.to_csv('film_writer_df.csv', index=False)
# film_ratings_df.to_csv('film_ratings_df.csv', index=False)
# film_awards_df.to_csv('film_awards_df.csv', index=False)